## Stacking Ensenble을 활용한 자율주행 센서 안테나 성능예측
> 팀명 : 될때까지간다리
>
> 작성일 : '22.08.31
>
> 개발환경 : Jupyter Notebook

In [1]:
# hyper parameter tuning을 위한 패키지 설치
!pip install optuna
!pip install catboost
!pip install skranger
!pip install ngboost
!pip install lightgbm
!pip install hyperopt

In [2]:
# 기본 modules
import pandas as pd
import random
import os
import warnings
warnings.filterwarnings('ignore')
import numpy as np
import tqdm

# 머신러닝 modules
from sklearn.model_selection import cross_val_score
from sklearn import metrics
from sklearn.model_selection import train_test_split, KFold
from sklearn.metrics import mean_squared_error
from sklearn.metrics import make_scorer
from sklearn.multioutput import MultiOutputRegressor
from sklearn.model_selection import KFold

from lightgbm import LGBMRegressor
from ngboost import NGBRegressor
from sklearn.svm import SVR
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor, ExtraTreesRegressor
from sklearn.tree import DecisionTreeRegressor
from xgboost import XGBRegressor
from sklearn.linear_model import ElasticNet, LinearRegression, Lasso, Ridge
from catboost import CatBoostRegressor, Pool
from skranger.ensemble import RangerForestRegressor
from sklearn.neighbors import RadiusNeighborsRegressor
from ngboost.scores import LogScore

from hyperopt import fmin, hp, tpe
from sklearn import metrics
from sklearn.model_selection import cross_val_score
from sklearn.metrics import make_scorer
from sklearn.multioutput import MultiOutputRegressor
from sklearn.model_selection import KFold
from sklearn.metrics import mean_squared_error
from sklearn.inspection import permutation_importance

# 모듈화된 함수 사용
import utils.preprocessing as preprocessing
import utils.utils as utils
import utils.stacking as stk

### Data Preprocessing

In [ ]:
ys = ['Y_01', 'Y_02', 'Y_03', 'Y_04', 'Y_05', 
      'Y_06', 'Y_07', 'Y_08', 'Y_09', 'Y_10', 
      'Y_11', 'Y_12', 'Y_13', 'Y_14']
ys_bounds = [[0.2, 2], [0.2, 2.1], [0.2, 2.1], 
             [7, 19], [22, 36.5], [-19.2, 19], 
             [2.4, 4], [-29.2, -24], [-29.2, -24],
             [-30.6, -20], [19.6, 26.6], [-29.2, -24],
             [-29.2, -24], [-29.2, -24]]

In [5]:
utils.seed_everything(utils.Config.seed)

train_df = pd.read_csv('Data/train.csv')
test_x = pd.read_csv('Data/test.csv')
train_x, train_y = preprocessing.dataset_split_X_y(train_df)

cols_with_zero_variance = preprocessing.zero_variance(train_x) # 분산이 0 (통과 여부)
train_x = train_x.drop(cols_with_zero_variance, axis = 1)
test_x = test_x.drop(cols_with_zero_variance, axis = 1)

train_x = train_x.drop(['X_10', 'X_11'], axis = 1) # 결측치가 많음 (결측치 = 0, 공지사항)
test_x = test_x.drop(['X_10', 'X_11'], axis = 1)

test_x = test_x.drop('ID', axis=1)

### Ensenble by LGBM

In [9]:
lg_1 = {'colsample_bytree': 0.572280100273023, 'learning_rate': 0.010283635038627429, 'max_depth': 180, 'min_child_samples': 135, 'min_split_gain': 0.04511227284338413, 'n_estimators': 900, 'num_leaves': 70, 'reg_alpha': 4.406681827912319, 'reg_lambda': 20.4785600448913, 'scale_pos_weight': 8.302374117433086, 'subsample': 0.1688669888026464}
lg_2 =  {'colsample_bytree': 0.7641322280477741, 'learning_rate': 0.010977205425053654, 'max_depth': 90, 'min_child_samples': 75, 'min_split_gain': 0.13379952895779884, 'n_estimators': 900, 'num_leaves': 80, 'reg_alpha': 1.9214119194170154, 'reg_lambda': 14.454450236504218, 'scale_pos_weight': 2.171961031806387, 'subsample': 0.9552593593877317}
lg_3 = {'colsample_bytree': 0.5504769098255781,  'learning_rate': 0.019653385015120244, 'max_depth': 220, 'min_child_samples': 25, 'min_split_gain': 0.1273611040963466, 'n_estimators': 470, 'num_leaves': 160, 'reg_alpha': 3.5549669150756706, 'reg_lambda': 39.88636182674132, 'scale_pos_weight': 12.46696320152359, 'subsample': 0.7590007450921917}
lg_4 = {'colsample_bytree': 0.5597537952569402, 'learning_rate': 0.02374663979814546, 'max_depth': 32, 'min_child_samples': 100, 'min_split_gain': 0.12211426885216736, 'n_estimators': 1263, 'num_leaves': 200, 'reg_alpha': 14.606693962963451, 'reg_lambda': 299.52278825209424, 'scale_pos_weight': 7.7785016838070735, 'subsample': 0.6254745287838821}
lg_5 = {'colsample_bytree': 0.4311015575880258, 'learning_rate': 0.01749725932551278, 'max_depth': 53, 'min_child_samples': 15, 'min_split_gain': 0.2820951740673634, 'n_estimators': 974, 'num_leaves': 165, 'reg_alpha': 9.604623064885754, 'reg_lambda': 12.314490508636432, 'scale_pos_weight': 6.6422956907936825, 'subsample': 0.7390190399971659}
lg_6 = {'colsample_bytree': 0.6889745043181079, 'learning_rate': 0.06146161938790444, 'max_depth': 89, 'min_child_samples': 10, 'min_split_gain': 0.669592868575692, 'n_estimators': 1169, 'num_leaves': 175, 'reg_alpha': 11.405277636150856, 'reg_lambda': 112.37954230084294, 'scale_pos_weight': 5.932435783263877, 'subsample': 0.8265223228903998}  
lg_7 = {'colsample_bytree': 0.8663251864650988, 'learning_rate': 0.018110306887688978, 'max_depth': 166, 'min_child_samples': 50, 'min_split_gain': 0.025403061552667243, 'n_estimators': 1080, 'num_leaves': 100, 'reg_alpha': 2.0131018839563666, 'reg_lambda': 63.56640846106552, 'scale_pos_weight': 1.8584564419776715, 'subsample': 0.7643028435523616}
lg_8 = {'colsample_bytree': 0.8970390757241629, 'learning_rate': 0.03571726260659087, 'max_depth': 164, 'min_child_samples': 30, 'min_split_gain': 0.2863362850926679, 'n_estimators': 740, 'num_leaves': 100, 'reg_alpha': 1.1167159754886287, 'reg_lambda': 280.9798636389436, 'scale_pos_weight': 4.75867892931176, 'subsample': 0.681716202670263}
lg_9 = {'n_estimators': 900, 'max_depth': 86, 'num_leaves': 150, 'min_child_samples': 85, 'colsample_bytree': 0.90507, 'subsample': 0.62362, 'min_split_gain': 0.21034, 'scale_pos_weight': 8.77311, 'reg_alpha': 0.07069, 'reg_lambda': 499.10672, 'learning_rate': 0.04679}
lg_10 = {'colsample_bytree': 0.8350973419202665, 'learning_rate': 0.03134966396365972, 'max_depth': 114, 'min_child_samples': 20, 'min_split_gain': 0.24406788869557822, 'n_estimators': 454, 'num_leaves': 115, 'reg_alpha': 1.0870546166564243, 'reg_lambda': 346.21163772786895, 'scale_pos_weight': 5.81617865285278, 'subsample': 0.45612075761336973}
lg_11 = {'colsample_bytree': 0.7285829045071064, 'learning_rate': 0.019839273085108612, 'max_depth': 71, 'min_child_samples': 50, 'min_split_gain': 0.35567737788276876, 'n_estimators': 970, 'num_leaves': 140, 'reg_alpha': 0.27353134227182774, 'reg_lambda': 157.85749037224548, 'scale_pos_weight': 5.956126991298146, 'subsample': 0.7509931500532172}
lg_12 = {'colsample_bytree': 0.6115826698158419, 'learning_rate': 0.010052927231718068, 'max_depth': 71, 'min_child_samples': 85, 'min_split_gain': 0.12003011548878659, 'n_estimators': 1300, 'num_leaves': 120, 'reg_alpha': 1.3013867029804251, 'reg_lambda': 269.3915696845848, 'scale_pos_weight': 5.290961082236748, 'subsample': 0.7542724715058367}
lg_13 = {'colsample_bytree': 0.9511047907962863, 'learning_rate': 0.023257873709858216, 'max_depth': 58, 'min_child_samples': 80, 'min_split_gain': 0.21488153574891886, 'n_estimators': 1300, 'num_leaves': 150, 'reg_alpha': 0.33761852089148814, 'reg_lambda': 57.05291849099506, 'scale_pos_weight': 2.0801436555772854, 'subsample': 0.5580106548214563}
lg_14 = {'colsample_bytree': 0.8851122740930837, 'learning_rate': 0.013136814152245062, 'max_depth': 249, 'min_child_samples': 65, 'min_split_gain': 0.2072264172906347, 'n_estimators': 450, 'num_leaves': 135, 'reg_alpha': 0.642890771203696, 'reg_lambda': 45.624663648443345, 'scale_pos_weight': 6.400746088779947, 'subsample': 0.30084274480143686}

In [10]:
params = []
params.append(lg_1)
params.append(lg_2)
params.append(lg_3)
params.append(lg_4)
params.append(lg_5)
params.append(lg_6)
params.append(lg_7)
params.append(lg_8)
params.append(lg_9)
params.append(lg_10)
params.append(lg_11)
params.append(lg_12)
params.append(lg_13)
params.append(lg_14)

cols = train_y.columns.tolist()

In [11]:
model = LGBMRegressor(n_jobs=-1, random_state=1, **lg_1)
model.fit(train_x, train_y['Y_01'])
pred1 = model.predict(test_x)

model = LGBMRegressor(n_jobs=-1, random_state=1, **lg_2)
model.fit(train_x, train_y['Y_02'])
pred2 = model.predict(test_x)

model = LGBMRegressor(n_jobs=-1, random_state=1, **lg_3)
model.fit(train_x, train_y['Y_03'])
pred3 = model.predict(test_x)

model = LGBMRegressor(n_jobs=-1, random_state=1, **lg_4)
model.fit(train_x, train_y['Y_04'])
pred4 = model.predict(test_x)

model = LGBMRegressor(n_jobs=-1, random_state=1, **lg_5)
model.fit(train_x, train_y['Y_05'])
pred5 = model.predict(test_x)

model = LGBMRegressor(n_jobs=-1, random_state=1, **lg_6)
model.fit(train_x, train_y['Y_06'])
pred6 = model.predict(test_x)


model = LGBMRegressor(n_jobs=-1, random_state=1, **lg_7)
model.fit(train_x, train_y['Y_07'])
pred7 = model.predict(test_x)


model = LGBMRegressor(n_jobs=-1, random_state=1, **lg_8)
model.fit(train_x, train_y['Y_08'])
pred8 = model.predict(test_x)

model = LGBMRegressor(n_jobs=-1, random_state=1, **lg_9)
model.fit(train_x, train_y['Y_09'])
pred9 = model.predict(test_x)

model = LGBMRegressor(n_jobs=-1, random_state=1, **lg_10)
model.fit(train_x, train_y['Y_10'])
pred10 = model.predict(test_x)

model = LGBMRegressor(n_jobs=-1, random_state=1, **lg_11)
model.fit(train_x, train_y['Y_11'])
pred11 = model.predict(test_x)

model = LGBMRegressor(n_jobs=-1, random_state=1, **lg_12)
model.fit(train_x, train_y['Y_12'])
pred12 = model.predict(test_x)

model = LGBMRegressor(n_jobs=-1, random_state=1, **lg_13)
model.fit(train_x, train_y['Y_13'])
pred13 = model.predict(test_x)

model = LGBMRegressor(n_jobs=-1, random_state=1, **lg_14)
model.fit(train_x, train_y['Y_14'])
pred14 = model.predict(test_x)

### csv파일로 만들기

In [12]:
sub = pd.read_csv('./sample_submission.csv')
sub

FileNotFoundError: [Errno 2] No such file or directory: './sample_submission.csv'

In [ ]:
sub['Y_01'] = pred1
sub['Y_02'] = pred2
sub['Y_03'] = pred3
sub['Y_04'] = pred4
sub['Y_05'] = pred5
sub['Y_06'] = pred6
sub['Y_07'] = pred7
sub['Y_08'] = pred8
sub['Y_09'] = pred9
sub['Y_10'] = pred10
sub['Y_11'] = pred11
sub['Y_12'] = pred12
sub['Y_13'] = pred13
sub['Y_14'] = pred14
sub.to_csv('./LGBM_Ensenble.csv',index=False)

In [ ]:
sub

,ID,Y_01,Y_02,Y_03,Y_04,Y_05,Y_06,Y_07,Y_08,Y_09,Y_10,Y_11,Y_12,Y_13,Y_14
0,TEST_00001,1.442924,1.273608,1.079165,14.338998,32.000107,16.832436,3.058966,-26.044237,-26.011586,-22.074013,24.653421,-25.961790,-25.905378,-26.023240
1,TEST_00002,1.515343,1.229354,1.148945,13.111977,31.132216,16.607473,3.188482,-26.104771,-26.140484,-22.357899,24.264452,-26.079527,-26.189696,-26.133548
2,TEST_00003,1.433870,1.137809,1.060110,14.745785,31.805783,16.971419,3.058863,-25.849074,-25.902935,-22.049387,24.602849,-25.795801,-25.811793,-25.814476
3,TEST_00004,1.437338,1.133470,1.005988,15.052939,32.461799,17.182728,3.094223,-25.627844,-25.679910,-21.813247,24.825474,-25.632177,-25.625208,-25.691007
4,TEST_00005,1.322911,0.984212,0.970184,14.883857,31.654056,16.943439,3.099447,-25.707569,-25.785620,-21.927005,24.699868,-25.668599,-25.506535,-25.665960
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
39603,TEST_39604,1.278604,0.960500,1.026256,12.603841,30.715649,16.790342,3.201161,-26.405848,-26.379649,-22.713770,24.486433,-26.295244,-26.276582,-26.320343
39604,TEST_39605,1.240492,0.834627,0.930362,14.287517,31.679095,16.864820,3.188203,-26.305844,-26.289248,-22.877738,24.348586,-26.297942,-26.166061,-26.304686
39605,TEST_39606,1.226990,0.920169,0.985180,12.999202,31.170677,16.673427,3.167141,-26.402755,-26.429248,-22.828484,24.195632,-26.400484,-26.403491,-26.456637
39606,TEST_39607,1.192636,0.833563,0.929844,14.065677,30.842401,16.801811,3.178882,-26.418736,-26.372937,-22.737590,24.475538,-26.301406,-26.334713,-26.399161


In [ ]:
sub.to_csv('LGBM_Ensenble.csv', index=False)

SyntaxError: ignored

### Result Mixing

In [ ]:
sub.to_csv('./stack_mid.csv', index=False)

In [ ]:
sub.to_csv('/content/drive/MyDrive/fin_one.csv', index=False)

In [ ]:
en_1 = pd.read_csv('/content/drive/MyDrive/submission_helpmodel.csv')

In [ ]:
en_ = en_1.iloc[:,1:] * 0.8
sub_ = sub.iloc[:, 1:] * 0.2
fin = en_ + sub_

In [ ]:
fin

,Y_01,Y_02,Y_03,Y_04,Y_05,Y_06,Y_07,Y_08,Y_09,Y_10,Y_11,Y_12,Y_13,Y_14
0,1.426577,1.213980,1.104271,14.136061,31.658121,16.815616,3.101089,-26.037361,-26.044660,-22.083655,24.530811,-25.924984,-25.968781,-26.009203
1,1.490870,1.223105,1.149351,12.998445,31.057604,16.522579,3.176718,-26.178148,-26.142853,-22.429383,24.285534,-26.100183,-26.117122,-26.130780
2,1.418025,1.125267,1.066391,14.676344,31.859178,16.870145,3.080107,-25.916346,-25.947726,-22.053972,24.589953,-25.885129,-25.798497,-25.883481
3,1.439449,1.147656,1.026739,15.216625,32.698535,17.198862,3.072755,-25.759455,-25.753470,-21.759636,24.803328,-25.609615,-25.616054,-25.634016
4,1.335854,1.002284,0.968494,14.969871,31.655982,16.947306,3.124099,-25.782356,-25.788519,-21.931106,24.643661,-25.592808,-25.676616,-25.690867
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
39603,1.269329,0.970946,1.006430,12.843846,31.056610,16.755690,3.162700,-26.356488,-26.310821,-22.830729,24.414755,-26.278037,-26.287378,-26.326881
39604,1.259122,0.876331,0.959379,14.343368,31.693058,16.860753,3.186614,-26.281565,-26.261941,-22.815099,24.420250,-26.245571,-26.237540,-26.275911
39605,1.236872,0.915140,0.965655,13.041076,30.985753,16.641565,3.155219,-26.438608,-26.396989,-22.884650,24.281932,-26.402059,-26.432641,-26.459169
39606,1.219312,0.859381,0.920601,13.654177,31.101830,16.857856,3.190817,-26.395030,-26.316000,-22.854529,24.480340,-26.334610,-26.348982,-26.391253


In [ ]:
id = sub.iloc[:,0]

In [ ]:
fin = pd.concat([id, fin], axis=1)
fin.to_csv('/content/drive/MyDrive/stack_mid_.csv', index=False)

In [ ]:
fin = en_1 * 0.8 + sub * 0.2
fin.to_csv('./help+stack_mid.csv', index=False)

TypeError: ignored